In [1]:
import scanpy as sc

from bio_grns import (
    Trajectory,
    GRNSimulator
)

In [2]:
### Construct a trajectory ###

trajectory_1 = Trajectory(
    500,                           ### Length of trajectory in time units (minutes)
    primary_trajectory = True,     ### Trajectory is primary (represents wall clock)
    force_positive = True,         ### Return absolute value of activity
    latent_network_sparsity = 0.2, ### Ratio of TFs which will be influenced by this trajectory
    name = "Development"           ### Name this trajectory
)

trajectory_1.add_pattern(
    'random', 0.5, 0.05, 0.2
) ### Random walk starting with an activity of 1 and steps between 0.05 and 0.2

trajectory_1.add_pattern(
    'cyclic', 0, 0.01, 0.5
) ### Cycle from zero to one (halfway through the trajectory) to zero again with random step sizes between 0.01 and 0.5

trajectory_1.add_pattern(
    'updown', .8, .2, (0.01, 0.5)
) ### Cycle from .8 to 1 (20% into the trajectory) to 0 with random step sizes between 0.01 and 0.5

### Construct a trajectory ###

trajectory_2 = Trajectory(
    300,                           ### Length of trajectory in time units (minutes)
    primary_trajectory = False,    ### Trajectory is primary (represents wall clock)
    force_positive = True,         ### Return absolute value of activity
    latent_network_sparsity = 0.2, ### Ratio of TFs which will be influenced by this trajectory
    name = "Cycle"                 ### Name this trajectory
)

trajectory_2.add_pattern(
    'cyclic', 0, 0.05, 0.2
)

trajectory_2.add_pattern(
    'cyclic', 0.33, 0.05, 0.2
)

trajectory_2.add_pattern(
    'cyclic', 0.66, 0.05, 0.2
)

In [3]:
### Create a simulator ###

sim = GRNSimulator(
    5000,                        ### Number of genes
    200,                         ### Number of genes which are regulators
    81,                          ### Random number for seeding generator
    n_samples = 1000,            ### Number of samples to generate
    counts_per_sample = 5000,    ### Number of read counts per sample
    debug=True
)

sim.add_trajectory(trajectory_1)   ### Add the trajectory to the simulator
sim.add_trajectory(trajectory_2)   ### Add the trajectory to the simulator

sim.set_biophysical_parameters(
    halflife_limits = (20, 200)    ### Set boundries for halflife (to calculate decay constant)
)
sim.set_network_parameters(
    regulatory_network_sparsity = 0.04,       ### Set the sparsity in the target -> regulator network
    regulatory_network_activator_ratio = 0.8  ### Set the ratio of positive target -> regulator network edges
)

sim.simulate()

2022-11-08 12:47:15,924 Generating trajectory -> regulator networks
2022-11-08 12:47:15,926 Generated (200, 3) network with 39 nonzero edges
2022-11-08 12:47:15,927 Generated network for Development
2022-11-08 12:47:15,928 Generated (200, 3) network with 39 nonzero edges
2022-11-08 12:47:15,928 Generated network for Cycle
2022-11-08 12:47:15,928 Simulating gene biophysical parameters
2022-11-08 12:47:15,929 Simulating regulatory network
2022-11-08 12:47:15,964 Generated (5000, 200) network with 39920 nonzero edges
2022-11-08 12:47:15,965 Simulating gene expression over time
2022-11-08 12:47:15,966 Generating pattern gene expression for Development
2022-11-08 12:47:15,966 Generating pattern regulatory activity for Development
2022-11-08 12:47:15,966 Generating pattern dynamic states for Development
2022-11-08 12:47:15,968 Generating dynamic expression (500 x 5000) with activation function relu_onemax
2022-11-08 12:47:16,181 Generating pattern gene expression for Cycle
2022-11-08 12:47:1

In [4]:
sim.save_network("benchmark_gold_standard.tsv")
sim.save_network("benchmark_prior_knowledge_network.tsv", edges_to_include=0.5)

for i in range(5):
    adata = sim.generate_count_data(sparse=True, no_metadata=True)
    adata.write(f"benchmark_count_data_{i}.h5ad")

2022-11-08 12:47:16,630 Downsampling network with 39920 edges by removing 19960 edges (0.5)
2022-11-08 12:47:16,926 Simulating count data from trajectories
100%|██████████| 1000/1000 [00:00<00:00, 1831.96it/s]
2022-11-08 12:47:17,594 Simulating count data from trajectories
100%|██████████| 1000/1000 [00:00<00:00, 1846.16it/s]
2022-11-08 12:47:18,269 Simulating count data from trajectories
100%|██████████| 1000/1000 [00:00<00:00, 1813.72it/s]
2022-11-08 12:47:18,948 Simulating count data from trajectories
100%|██████████| 1000/1000 [00:00<00:00, 1817.21it/s]
2022-11-08 12:47:19,625 Simulating count data from trajectories
100%|██████████| 1000/1000 [00:00<00:00, 1832.28it/s]


In [ ]:
import pandas as pd

net = pd.read_csv('')